In [1]:
import numpy as np
import cv2

In [ ]:
class DemosaicingVNG:
    def __init__(self, k1=1.5, k2=0.5):
        self.k1 = k1
        self.k2 = k2

    def demosaic(self, img):
        for row in range(2, img.shape[0] - 2):
            for col in range(2, img.shape[1] - 2):
                matrix = img[row-2:row+2, col-2:col+2]
                gradients = DemosaicingVNG._calc_gradients(matrix)
                threshold = DemosaicingVNG._calc_threshold(gradients,
                                                           self.k1,
                                                           self.k2) 
                directions = [key for key, value in gradients.items() if value < threshold]
    
    @staticmethod
    def _calc_threshold(gradients, k1, k2):
        gradient_min = min(gradients.values())
        gradient_max = max(gradients.values())
        return k1 * gradient_min + k2 * (gradient_max - gradient_min)
    
    @staticmethod
    def _calc_gradients(matrix):
        red, green, blue = matrix
        gradients = {}
        for row_offset in [-1, 0, 1]:
            for col_offset in [-1, 0, 1]:
                if row_offset == 0 or col_offset == 0:
                    gradient = abs(green[2 + row_offset, 2 + col_offset] - 
                                   green[2 - row_offset, 2 - col_offset])
                    gradient += abs(red[2 + row_offset * 2, 2 + col_offset * 2] -
                                    red[2 - row_offset * 2, 2 - col_offset * 2])
                    gradient += abs(blue[2 + row_offset, 2 + col_offset] -
                                    blue[2 - row_offset, 2 - col_offset]) / 2
        return gradients